In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 

pipe_svc = make_pipeline(StandardScaler(),SVC(random_state = 1))

param_range = [0.0001,0.001, 0.01, 0.1, 1, 10, 100,1000, 10000, 100000]
param_grid =[{'svc__C':param_range,
              'svc__kernel':['linear']},
              {'svc__C':param_range,
               'svc__gamma':param_range,
               'svc__kernel':['rbf']}]
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, scoring='accuracy',cv=10,refit=True,n_jobs = -1)
gs.fit(X_train,y_train)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 

pipe_svc = make_pipeline(StandardScaler(),SVC(random_state = 1))

param_range = [0.0001,0.001, 0.01, 0.1, 1, 10, 100,1000]
param_grid =[{'svc__C':param_range,
              'svc__kernel':['linear']}]

gs = GridSearchCV(estimator=pipe_svc,param_grid=param_grid, scoring = 'accuracy',cv=2)
scores = cross_val_score(gs,X_train,y_train,scoring='accuracy',cv=5)
print('Точность перекрестной проверкиЖ %.3f +/- %.3f'%(np.mean(scores),np.std(scores)))


In [ ]:
import ds
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

score = 0
for i in range(30):
    df_train = df.sample(random_state = i, frac = 0.8)
    df_test = df.drop(df_train.index)

    pipe_knn = make_pipeline(Row_selector((df['CryoSleep']==0)&(df['Age']>12.5)), StandardScaler(),ds.LDA(plot=False),KNeighborsClassifier(n_neighbors =20, p=1,weights ='uniform'))
    pipe_tree = make_pipeline(Row_selector((df['CryoSleep']==0)&(df['Age']>12.5)), StandardScaler(),ds.LDA(plot=False),DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1))

    estimators = [
        ('knn pipeline', pipe_knn),                       
        ('tree pipeline', pipe_tree)
    ]

    final_estimator = LogisticRegression()
    #DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1)#DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=1)#SVC(kernel = 'rbf', random_state = 1, gamma = 10, C =100)

    pipe = StackingClassifier(estimators=estimators, final_estimator = final_estimator,passthrough = True)
    pipe = pipe.fit(df_train[features],df_train["Transported"])
    score +=pipe.score(df_test[features],df_test["Transported"])
score /=30
score


In [ ]:
#{'condition': conditions, 'pipeline': scikitlearn_pipeline}
class Model_pipeline:
    def __init__(self, input_conveyer):
        self.conveyer = input_conveyer
    
    def fit(self,df,check_column):
        for part in self.conveyer:
            dz = df.drop(~part['condition'], axis =1)
            features = dz.columns
            features.remove(check_column)
            X_train = dz[features]
            y_train = dz[check_column]
            part['pipeline'].fit(X_train, y_train)

    def score(self, df_test,check_column):
        for part in self.conveyer:
            dz = df_test.drop(~part['condition'], axis =1)
            features = dz.columns
            features.remove(check_column)
            X_test = dz[features]
            y_test = dz[check_column]
            part['pipeline'].fit(X_test, y_test)


In [ ]:
import ds
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),LogisticRegression(penalty = 'l2',random_state = 1,solver = 'lbfgs', max_iter = 1000))
pipe_tree = make_pipeline(StandardScaler(),ds.LDA(plot = False),DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=1))

estimators = [
    ('Logistic Regression on full space', pipe_lr),                       
    ('Tree on LDA space', pipe_tree)
]

final_estimator = LogisticRegression(penalty = 'l2',random_state = 1,solver = 'lbfgs', max_iter = 1000)#DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=1)

pipe = StackingClassifier(estimators=estimators, final_estimator = final_estimator,passthrough = False)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer

scorer = make_scorer(f1_score, pos_label=-1)

print('Точность: %.3f'% precision_score(y_true=y_test, y_pred = y_pred))
print('Полнота: %.3f'% recall_score(y_true=y_test, y_pred = y_pred))
print('Мера F1: %.3f'% f1_score(y_true=y_test, y_pred = y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = pipe.predict(X_test)
confmat = confusion_matrix(y_true = y_test, y_pred = y_pred)
print(confmat)

tp = confmat[0][0]
fn = confmat[0][1]
fp = confmat[1][0]
tn = confmat[1][1]

print(tp, fn)
print(fp, tn)

ERR = (fp+fn)/(tp+fp+fn+tn)
ACC = 1-ERR

FPR = fp/(fp+tn)
TPR = tp/(tp+fn)

PRE = tp/(tp+fp)
REC = tp/(fn+tp)

print('Точность (PRE) =%.3f'% PRE, 'Полнота (REC) = %.3f'% REC)
print('Точность по всем классам: %.3f'% ACC)

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from numpy import interp

pipe = make_pipeline(StandardScaler(),LogisticRegression(penalty = 'l2',random_state = 1,solver = 'lbfgs', max_iter = 1000, C=100.0))

pipe_tmp = pipe.fit(X_train,y_train)
probas = pipe_tmp.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test,probas[:,1],pos_label = 1)

lr_auc = roc_auc_score(y_test, probas[:,1])
print('LogisticRegression: ROC AUC=%.3f' % (lr_auc))

roc_auc = auc(fpr,tpr)

plt.plot(fpr,tpr,label = 'ROC кривая (прощадь = %0.2f)'% roc_auc)

plt.plot([0,1], [0,1], linestyle = '--', color = (0.6, 0.6, 0.6), label='случайное угадывание')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('доля ложноположительных классификаций')
plt.ylabel('дооля истинно положитльных клаассификаций')
plt.legend(loc = "lower right")
plt.show()